# CNS Conservatory Tree generation

## STEP 1: Generate separate files for every unique ortholog in list

In [ ]:
import pandas as pd
import glob
import os
# set family name
family = "Malvaceae"
os.chdir(family) #this navigates the directory to the family folder
!ls # list folder contents

In [ ]:
df = pd.read_csv(family + '_CRE_inall_unique.csv') # import csv with pandas and display
df

In [ ]:
df = df[["RefGeneID", "SpeciesGeneId", "uniqueSpecies"]] # just keep relevant columns
df

In [ ]:
df = df.pivot(index='RefGeneID', columns='uniqueSpecies', values='SpeciesGeneId')
df

In [ ]:
df.to_csv('Family_gene_table.csv') #write new table to csv
!ls # list dir contents

In [ ]:
# split Family_gene_table.csv into separate files - one file for each gene in the table.
!split -l 1 -a 3 Family_gene_table.csv locus_ 

In [ ]:
!rm locus_aaa #remove the txt file created from the header

In [ ]:
# check that the number of files matches the number of genes. Pandas does not count the header as a row in the dataframe.
if (len(df) != len(glob.glob("locus_*"))):
    print("Problem: csv rownumber and generated file number do not match. ", len(df), ", ", len(glob.glob("locus_*")))
else:
    print("Same number: ", len(df), ", ", len(glob.glob("locus_*")))

In [ ]:
# move all files to a new directory - 'LocusLists' within family folder
!mkdir LocusLists
!mv locus_* LocusLists

In [ ]:
#change directories to LocusLists and check
os.chdir('LocusLists')
!ls

In [ ]:
# Rename all the resulting gene files with a number and a .txt extension
!n=1;
!for f in *; do mv "$f" "locus_$((n++)).txt"; done
!ls

In [ ]:
# each file needs commas replaced with /n, " characters removed, and > added to the beginning of every line
!for file in *.txt; do sed 's/,/\n/g' "$file" > "$file.1"; sed 's/^/\>/' "$file.1" > "$file.2"; sed 's/^M//g' "$file.2" > "$file.3" ; done

# get rid of backup files
!rm *.txt
!rm *.txt.1
!rm *.txt.2

# remove .3 extension on final file
!for file in *.txt.3; do mv -- "$file" "${file%.txt.3}.txt"; done

# check that every file has the right number of taxa (refer to species list)
!grep -c -F ">" *.txt > taxon_counts.txt

In [ ]:
# optional: sometimes adding the ">" at the beginning of each txt file causes no match to protein library. Removed for Fabaceae.
!for file in *.txt; do sed -i 's/[>]//g' $file; done

In [ ]:
# check that everything looks good by looking at some random files and checking counts
!cat locus_1.txt
!cat locus_888.txt

!cat taxon_counts.txt

In [ ]:
#move files to peptide library folder
#!mkdir ../Rosaceae_PeptideLibraries
!cp locus_* ../Malvaceae_PeptideLibraries

## STEP 2: Search for all loci in peptide fasta files

### Transferring files directly from the Marlin server to the UMass cluster  

* Step 1: On the UMass cluster terminal, connect to Marlin using sftp and navigate to peptide folder

`sftp username@marlin.bio.umass.edu`  
`cd ../../../data/bartlett/Conservatory/peptides`  
`ls` to list all family folders  
`cd Fabaceae` #example folder with peptide files on Marlin  

* Step 2: Set the local directory where you want to download the files using the lcd command  

note: lcd works exactly like cd. If you are unsure where you are in your local directory, you can us lls, which works the same as ls.  
`lcd CNS/Fabaceae/Fabaceae_PeptideLibraries` #example local destination folder on UMass cluster

* Step 3: Copy everything over using mget

`mget *` #transfers over each file in the directory


In [ ]:
# change directory to peptide library
os.chdir('../Malvaceae_PeptideLibraries')
!ls
#os.chdir(family + "_PeptideLibraries")

In [ ]:
# convert fasta files to have all amino acids on 1 line
!ls
!for f in *.fasta; do dos2unix $f; done
!for f in *.fasta; do awk '/^>/ {printf("\n%s\n",$0);next; } { printf("%s",$0);} END {printf("\n");}' $f > $f.fullfile.1line.fa; done


In [ ]:
# check that it worked
!ls
!head Bceiba.fixed.proteins.fasta
!head Bceiba.fixed.proteins.fasta.fullfile.1line.fa

In [ ]:
#removed -w for whole match
#for each locus text file, match the gene name to the protein sequence in the fasta file. Copy the ammino acids over.
!for f in *.txt; do grep -A 1 -m 1 -f $f -h *.fa > $f.fasta; done

In [ ]:
#Identify genes with correct number of matches and copy good loci to a new folder. Change "target" variable number.
os.mkdir("GoodLoci")

In [ ]:
%%script bash
    target=20
    allfiles=$(ls *.txt.fasta | wc -l)
    wrong=0
    echo "Total files: $allfiles"
    for f in *.txt.fasta
    do
        count=$(grep -o '>' $f | wc -l)
        if [ $count != $target ]
        then
            wrong=$((wrong + 1))
        else
        cp $f GoodLoci
        fi
    done
echo "Wrong: $wrong"
echo "Good with $target matches: $((allfiles - wrong))"

In [ ]:
#os.chdir('Malvaceae')
!mkdir LociForAlignment
!ls

In [ ]:
os.chdir("Malvaceae_PeptideLibraries/GoodLoci")
!mv *.txt.fasta ../../LociForAlignment/

In [ ]:
os.chdir('../../LociForAlignment')
!ls

In [ ]:
#remove '--' marks from grep
!for file in *; do sed -i 's/--//g' $file; done

In [ ]:
#check that it worked (no '--')
!cat locus_1.txt.fasta

## STEP 3: align using MAFFT & replace gene names with species names

#### Step 1: Start a new interactive session from the terminal. Make sure you are in the LociForAlignment directory.

`bsub -Is -q interactive -W 8:00 -n 1 -R "span[hosts=1]" bash`

#### Step 2: Run mafft on all fasta files in the LociForAlignment directory

`module load MAFFT/7.313`

`for f in *.fasta; do mafft-linsi $f > $f.aln; done`

### While MAFFT is running, identify and prep gene ids for species name replacement

In [ ]:
#go to species family directory
os.chdir("../")
!ls

In [ ]:
df = pd.read_csv("Family_gene_table.csv")
df

In [ ]:
#make a new dataframe 'newdf' to store geneids/nicknames
df = pd.read_csv("Family_gene_table.csv")
data = df.columns.values.tolist()
newdf = pd.DataFrame(data)
newdf.rename(columns={newdf.columns[0]: "Genome name"}, inplace=True)
newdf["nicknames"] = ""
newdf

In [ ]:
#for each species column in the unique_species.csv, extract the first 4 characters.
#important note: if three characters is not enough to have a unique identifier for each species, increase to 5.

nickname_data = []
for column in df:
    df[column] = df[column].astype('string').str[0:4]
    df[column] = df[column].astype('category')
    nickname = df[column].cat.categories.values.tolist()
    nickname_data.append(nickname)
print(nickname_data)

In [ ]:
#check that no values are duplicated, all should be false
check_dup_df = pd.DataFrame(nickname_data)
check_dup_df.duplicated()

In [ ]:
newdf["nicknames"] = nickname_data
newdf

In [ ]:
newdf.to_csv("uniqueSpeciesNicknames.csv")

#delete header from csv file
!sed -i -e 1,1d uniqueSpeciesNicknames.csv

#check that the four letter geneid doesnt match anything but itself or its species in the final csv

In [ ]:
#remove extra characters
!sed -i 's/"//g' uniqueSpeciesNicknames.csv
!sed -i 's/\[//g' uniqueSpeciesNicknames.csv
!sed -i 's/\]//g' uniqueSpeciesNicknames.csv
!sed -i "s/'//g" uniqueSpeciesNicknames.csv
!sed -i "s/ //g" uniqueSpeciesNicknames.csv

In [ ]:
%%script bash
    cat uniqueSpeciesNicknames.csv | while read line;
    do
        #line="$(sed -n '10p' uniqueSpeciesNicknames.csv)"
        #echo "The line is: $line"
        mapfile -t my_array < <( for i in ${line//,/ }; do echo "$i"; done )
        #echo "The array is: ${my_array[@]}"
        echo "Genome species name is: ${my_array[1]}"
        length="${#my_array[@]}"
        echo "GeneID identifiers are:"
        for (( j=2; j<length; j++ )); do echo "${my_array[$j]}"; done
    done

### When MAFFT is finished, replace gene ids

In [ ]:
#check that it worked (should see .aln files)
os.chdir("LociForAlignment")
!ls

In [ ]:
#check .aln file
!cat locus_3.txt.fasta.aln

In [ ]:
#make a new Test directory to modify species names
os.mkdir('../Test')
#copy .aln files over
!cp *.aln ../Test/

In [ ]:
os.chdir('Test')
!ls

In [ ]:
#make sure you are in the right directory!
!cat ../uniqueSpeciesNicknames.csv

In [ ]:
#replace all geneids with species names in the Test folder
!ls

In [ ]:
#%%script bash
#    line="$(sed -n '4p' ../uniqueSpeciesNicknames.csv)"
#    echo "$line"
#    mapfile -t my_array < <( for i in ${line//,/ }; do echo "$i"; done )
#    geneid="${my_array[2]}"
#    name="${my_array[1]}"
#    echo "Replacing $geneid with $name"
#    sed -i "s/>$geneid.*/>$name/g" locus_0.txt.fasta.aln

In [ ]:
#replace gene names with Species names in Test folder

In [ ]:
%%script bash
    cat ../uniqueSpeciesNicknames.csv | while read line;
    do
        mapfile -t my_array < <( for i in ${line//,/ }; do echo "$i"; done )
        length="${#my_array[@]}"
        for (( j=2; j<length; j++ ))
        do
            geneid="${my_array[j]}"
            name="${my_array[1]}"
            echo "Replacing $geneid with $name"
            for file in *; do sed -i "s/>$geneid.*/>$name/g" $file; done
        done
    done


In [ ]:
#check that each species appears once per file, no output is good

In [ ]:
%%script bash
    cat ../uniqueSpeciesNicknames.csv | while read line;
    do
        mapfile -t my_array < <( for i in ${line//,/ }; do echo "$i"; done )
        for file in *
        do
            count=$(grep -o "${my_array[1]}" $file | wc -l)
            if [ $count != 1 ]
            then
                echo $file; echo ${my_array[1]}; echo $count
            fi
        done
    done

In [ ]:
os.chdir('../Test')
!ls

In [ ]:
##replace * and . characters in fasta files
!for file in *; do sed -i 's/\.//g' $file; done
!for file in *; do sed -i 's/\*//g' $file; done

In [ ]:
#check that it worked
!cat locus_1.txt.fasta.aln

## STEP 4: Concatenate all good loci, run RAxML analysis


In [ ]:
#make sure you are in the Test directory
os.chdir ("../")
os.chdir("Test")
!ls

In [ ]:
#Shuffle sets of 100, 200, and 300 random Loci and store in new folders. 
#This is because depending on dataset we might have to use less loci than 200, or we might be able to use more.
#os.mkdir("Shuffle_Set100")
os.mkdir("Shuffle_Set600")
os.mkdir("Shuffle_Set300")
#!cp $(ls *.aln | shuf -n 100) Shuffle_Set100
!cp $(ls *.aln | shuf -n 600) Shuffle_Set600
!cp $(ls *.aln | shuf -n 300) Shuffle_Set300

In [ ]:
#Or just Shuffle all if you dont have a lot of loci
os.mkdir("Shuffle_Set_all")
!cp *.aln Shuffle_Set_all

In [ ]:
#Concatenate shuffle sets with catfasta2phyml.pl
#!perl ../../catfasta2phyml.pl Shuffle_Set100/* > Shuffle_Set100.fa
!perl ../../catfasta2phyml.pl Shuffle_Set600/* > Shuffle_Set600.fa
!perl ../../catfasta2phyml.pl Shuffle_Set300/* > Shuffle_Set300.fa

In [ ]:
!perl ../../catfasta2phyml.pl Shuffle_Set_all/* > Shuffle_Set_all.fa

In [ ]:
#check ShuffleSet files
!less Shuffle_Set300.fa

In [ ]:
#copy ShuffleSets to Final Concatenated folder and run raxml. I could only get this to work with raxml-ng. Details: https://isu-molphyl.github.io/EEOB563/computer_labs/lab4/raxml-ng.html
#os.mkdir("../FinalConcatenated")
#!cp Shuffle_Set_all.fa ../FinalConcatenated
#!cp Shuffle_Set100.fa ../FinalConcatenated
!cp Shuffle_Set600.fa ../FinalConcatenated
!cp Shuffle_Set300.fa ../FinalConcatenated

In [ ]:
os.chdir("../FinalConcatenated")
!ls

In [ ]:
#check that the files were copied
!head Shuffle_Set600.fa

### Running & optimizing large alignments with raxml-ng
#### Step 1: Load raxml-ng from terminal with command 
`module load raxml-ng/0.9.0`
#### Step 2: Run raxml-ng in 'parse' mode first. Submit the following job in the terminal with your .fa or .phy file

`bsub -q interactive -Is -W 1:00 -n 1 -R "span[hosts=1]" singularity exec $RAXMLNGIMG raxml-ng-mpi --msa Shuffle_Set200.fa --model JTT+G --parse --prefix parse200`

* This will create a log file that you can check to see the optimized number of threads. It also has information about memory requirements.  
* You can run parse on different numbers of concatenated loci (the different shuffle sets) to see what is best time-wise. You can possibly use more than 300 loci if there is a high percentage of invariant sites and a low number of species.
* --model JTT+G specifies amino acid alignment. More detail [here](https://github.com/amkozlov/raxml-ng/wiki/Input-data).

#### Step 3: Run raxml-ng with updated thread number

* Once you know how many threads you want to use, change the numbers in the command below at '-n X' and '--threads X', where X equals your number of desired threads. I double checked with the ghpcc system admin people and this is the most efficient way to run the program. More detail [here](https://github.com/amkozlov/raxml-ng/wiki/Parallelization#so-how-many-threadscores-should-i-use).

`bsub -q long -W 8:00 -n 32 -o "raxml_job.out" -e "raxml_job.err" -R "span[hosts=1]" -R "rusage[mem=2000]" singularity exec $RAXMLNGIMG raxml-ng-mpi --msa Shuffle_Set200.fa --model JTT+G --threads 32`

#### Example runtimes:

* For 200 concatenated Loci of Rutaceae (w/ 10 species), there were 92% invariant sites, 80MB for the estimated memory requirements and 11 threads recommended. I ran this on 1 thread with 2000MB and it only took 15 mins to complete. 
* For 100 concatenated Loci of Malvaceae (w/ 19 species), there were 60% invariant sites, 1000MB estimated memory and 39 threads recommended. This took 3 hours to complete on 32 threads.


## Step 6: Looking at generated ML tree

In [ ]:
#install toytree to look at trees in notebook
!pip install toytree

In [ ]:
os.chdir("../../Cucurbitaceae/FinalConcatenated")
!ls

In [ ]:
import sys
#tell python to look in your local folder for the newly installed packages
sys.path.append('/home/ad32a/.local/lib/python3.6/site-packages')

In [ ]:
#import the new packages to python
import toytree
import toyplot
import numpy as np

In [ ]:
#load in the new tree
tre = toytree.tree("Shuffle_Set300.fa.raxml.bestTree")
#load in the new tree

In [ ]:
#draw tree
tre.unroot().write()

In [ ]:
!cat Malvaceae_Shuffle_Set3.fa.raxml.bestTree

In [ ]:
tre.draw(tip_labels_align=True, node_labels='support');

In [ ]:
tre3 = toytree.tree("(((((Cor,Omo),(Tca,Hum)),Dzi),(Kae,Gwh)),((Gra,Ghi,Gda,Gla,Gto,Mba),(Gmu,Glo,Gtr,Gsc,Kab,Gar)));")
tre3.draw(tip_labels_align=True);

In [ ]:
text_file = open("constraint.txt", "w")
n = text_file.write("(((((Cor,Omo),(Tca,Hum)),Dzi),(Kae,Gwh)),((Gra,Ghi,Gda,Gla,Gto,Mba),(Gmu,Glo,Gtr,Gsc,Kab,Gar)));")
text_file.close()

In [ ]:
tre4 = toytree.tree("constraint.txt")
tre4.draw(tip_labels_align=True);

In [ ]:
label_list = "1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19"
tre.draw(tip_labels=label_list)

In [ ]:
merged_df

In [ ]:
label_list3 = "Gossypium armourianum","Gossypium barbadense","Gossypium tomentosum","Gossypium darwinii","Gossypium lobatum","Gossypium mustelinum","Gossypium schwendimanii", "Gossypium arboreum", "Gossypium laxum", "Gossypium hirsutum","Gossypium trilobum", "Gossypium aridum" , "Hibiscus cannabinus", "Hibiscus syriacus", "Durio zibethinus", "Corchorus capsularis", "Corchorus olitorius","Herrania umbratica","Theobroma cacao" 

In [ ]:
tre.draw(tip_labels=True)

In [ ]:
tre.draw(tip_labels=True)
tre = tre.root(wildcard="Tca") #root the tree
tre.draw(tip_labels=label_list3)

In [ ]:
canvas, axes, mark = tre.draw(width=700, height=500, tip_labels=True, tip_labels_align=True,use_edge_lengths=False)

In [ ]:
label_list3 = "Herrania umbratica","Theobroma cacao","Corchorus capsularis","Corchorus olitorius","Durio zibethinus","Hibiscus cannabinus","Hibiscus syriacus","Gossypium aridum","Gossypium armourianum","Gossypium barbadense","Gossypium tomentosum","Gossypium darwinii","Gossypium lobatum","Gossypium mustelinum","Gossypium schwendimanii","Gossypium trilobum","Gossypium arboreum","Gossypium laxum","Gossypium hirsutum"

In [ ]:
import toyplot.pdf
toyplot.pdf.render(canvas, "Cucurbitaceae_Best_ML_tree.pdf")

In [ ]:
tre3 = toytree.tree("FINALTREE2.raxml.bestTree")
tre3.draw(tip_labels=True)
canvas, axes, mark = tre3.draw(width=1400, height=500, tip_labels=True)